In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = './input/'
train = pd.read_csv(path + 'train.csv')

** Cabin has too many missing values, drop it**

In [3]:
print(train.shape)
train.isnull().sum()

(891, 12)


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
train.drop('Cabin', axis=1,inplace = True)

** Inspired by Guillaume Martin, create feature - title.** <br>
** Fill missing age with average age of each title.** 

In [5]:
def title_convert(x):
    if x in ['Mlle','Ms']:
        return 'Miss'
    elif x in ['Mme']:
        return 'Miss'
    elif x in ['Jonkheer','Don','Dr','Capt','Col','Major']:
        return 'Sir'
    elif x in ['the Countess','Dona']:
        return 'Lady'
    else:
        return x

train['title'] = train.Name.apply(lambda x: re.compile(", (.*?)\.").findall(x)[0])
train.title = train.title.apply(title_convert)

In [6]:
title_age_mean= train.groupby(['title','Sex']).Age.mean()
title_age_mean

title   Sex   
Lady    female    40.500000
Master  male       4.574167
Miss    female    21.860000
Mr      male      32.368090
Mrs     female    35.898148
Rev     male      43.166667
Sir     female    49.000000
        male      47.153846
Name: Age, dtype: float64

In [7]:
def missing_age(x):
    if np.isnan(x['Age']):
        for i, age in title_age_mean.iteritems():
            if i[0] == x['title'] and i[1] == x['Sex']:
                return age
    else:
        return x['Age']

In [8]:
train.Age = train.apply(missing_age, axis =1)

** Fill missing Embarked with mode**

In [9]:
train.Embarked.fillna(train.Embarked.mode()[0], inplace = True)

**Convert Parch and SibSp into wheather its value > 0 or not**

In [10]:
train['SibSp_True'] = train['SibSp']>0
train['Parch_True'] = train['Parch']>0

** Drop unnecessary columns**

In [11]:
train.drop(['PassengerId','Name','SibSp','Parch','Ticket'], axis=1,inplace = True)

In [12]:
train.head(5)

,Survived,Pclass,Sex,Age,Fare,Embarked,title,SibSp_True,Parch_True
0,0,3,male,22.0,7.2500,S,Mr,True,False
1,1,1,female,38.0,71.2833,C,Mrs,True,False
2,1,3,female,26.0,7.9250,S,Miss,False,False
3,1,1,female,35.0,53.1000,S,Mrs,True,False
4,0,3,male,35.0,8.0500,S,Mr,False,False


** These are the features will be used for modeling, and categorical features will be tranformed by one-hot encoding.**